Method 1: Direct Copy with Progress Bar

This code block allows you to copy the contents of a shared Google Drive folder (which you have added as a shortcut to 'My Drive') to a new destination within your Google Drive. It includes a progress bar for large transfers.

In [ ]:
from google.colab import drive
import os
import shutil
from tqdm import tqdm  # Library to show a Progress Bar

# 1. Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# --- Change the names here ---

source_folder = "Your Source Folder Name"
destination_folder = "Your Destination Folder Name"

# Path setup
source_path = f"/content/drive/MyDrive/{source_folder}"
destination_path = f"/content/drive/MyDrive/{destination_folder}"

# Use this list to skip Google Native Files
SKIP_EXTENSIONS = ('.gsheet', '.gdoc', '.gslides', '.gform', '.gdraw', '.gmap')

# -----------------------------------------------

def get_size_and_count(start_path):
    total_size = 0
    total_files = 0
    print("Calculating folder size... (Please wait)")

    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            # Skip checking size for Google native files (to avoid errors)
            if f.endswith(SKIP_EXTENSIONS):
                continue

            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                try:
                    total_size += os.path.getsize(fp)
                    total_files += 1
                except OSError:
                    pass # Ignore small errors when calculating size
    return total_size, total_files

# Let's start the work
try:
    if not os.path.exists(source_path):
        raise FileNotFoundError(f"Source folder not found: {source_path}")

    size_in_bytes, file_count = get_size_and_count(source_path)
    size_in_gb = size_in_bytes / (1024 * 1024 * 1024)

    print(f"\n📂 Folder Info (Excluding Google Docs/Sheets):")
    print(f"   - Total Size: {size_in_gb:.2f} GB")
    print(f"   - Total Files: {file_count}")
    print("-" * 30)

    if not os.path.exists(destination_path):
        print(f"🚀 Creating new folder: {destination_path}")
        os.makedirs(destination_path)
    else:
        print(f"ℹ️ Folder exists. Resuming copy to: {destination_path}")

    pbar = tqdm(total=file_count, unit='file', desc='Syncing', mininterval=0.5)

    skipped_count = 0
    google_files_skipped = 0
    error_skipped = 0

    for dirpath, dirnames, filenames in os.walk(source_path):
        rel_path = os.path.relpath(dirpath, source_path)
        dest_dir = os.path.join(destination_path, rel_path)

        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)

        for file in filenames:
            src_file = os.path.join(dirpath, file)
            dst_file = os.path.join(dest_dir, file)

            # --- 1. Google File Check ---
            # Check if this file is a Google Doc/Sheet
            if file.endswith(SKIP_EXTENSIONS):
                google_files_skipped += 1
                # Google files are not added to the progress bar, so no need to update.
                continue

            # --- 2. Existing File Check ---
            if os.path.exists(dst_file):
                skipped_count += 1
                pbar.update(1)
                continue

            # --- 3. Copy Attempt with Error Handling ---
            try:
                shutil.copy2(src_file, dst_file)
                pbar.update(1)

            except OSError as e:
                # If Errno 95 or another issue occurs, skip without stopping.
                if e.errno == 95 or "Operation not supported" in str(e):
                    # This might be a Google file without an extension.
                    error_skipped += 1
                else:
                    print(f"\n⚠️ Error copying {file}: {e}")
                    error_skipped += 1

                # Even if an error occurs, update the bar to avoid getting stuck.
                pbar.update(1)

    pbar.close()
    print(f"\n✅ Success! Job Done.")
    print(f"📊 Summary:")
    print(f"   - Copied or Exists: {file_count - error_skipped}")
    print(f"   - Already Existed (Skipped): {skipped_count}")
    print(f"   - Google Docs/Sheets Skipped: {google_files_skipped + error_skipped}")

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

Method 2: Using API for Shortcut Creation and Copy

This code block provides an alternative method to copy a shared Google Drive folder directly from its link to your 'My Drive'. It authenticates with the Drive API to create a temporary shortcut, then copies the contents with a progress bar, and finally cleans up the temporary shortcut.

In [ ]:
import os
import shutil
import re
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build
from tqdm import tqdm
import time

# --- Settings ---

# 1. Put your Shared Link here
SHARED_LINK = "YOUR_SHARED_DRIVE_LINK_HERE"

# 2. Name of the folder to save
DESTINATION_FOLDER_NAME = "Your Destination Folder Name"

# 3. Path (do not change)
BASE_PATH = "/content/drive/MyDrive/"

# Use this list to skip Google Files
GOOGLE_WORKSPACE_EXTENSIONS = (".gdoc", ".gsheet", ".gslides", ".gdraw", ".gform", ".gmap")

def extract_id_from_link(link):
    patterns = [r'folders/([a-zA-Z0-9-_]+)', r'id=([a-zA-Z0-9-_]+)']
    for pattern in patterns:
        match = re.search(pattern, link)
        if match:
            return match.group(1)
    return None

def create_shortcut(service, file_id, shortcut_name):
    file_metadata = {
        'name': shortcut_name,
        'mimeType': 'application/vnd.google-apps.shortcut',
        'shortcutDetails': {
            'targetId': file_id
        }
    }
    shortcut = service.files().create(body=file_metadata, fields='id').execute()
    return shortcut.get('id')

def get_size_and_count(start_path):
    total_size = 0
    total_files = 0
    print("Calculating size... (Scanning files)")
    for dirpath, dirnames, filenames in os.walk(start_path):
        for f in filenames:
            # Google Files are not counted
            if f.lower().endswith(GOOGLE_WORKSPACE_EXTENSIONS):
                continue

            fp = os.path.join(dirpath, f)
            if not os.path.islink(fp):
                total_size += os.path.getsize(fp)
                total_files += 1
    return total_size, total_files

# --- Main Program Start ---

temp_shortcut_id = None
temp_shortcut_name = None
source_path = None

print("🔑 Authenticating User...")
auth.authenticate_user()
drive_service = build('drive', 'v3')

print("🔌 Mounting Drive...")
drive.mount('/content/drive', force_remount=True)

try:
    # 1. Get the ID
    folder_id = extract_id_from_link(SHARED_LINK)
    if not folder_id:
        raise Exception("Invalid Link! The link is incorrect.")

    print(f"🆔 Folder ID Found: {folder_id}")

    # 2. Create the Shortcut
    temp_shortcut_name = "TEMP_COPY_SHORTCUT_" + folder_id[:5]
    print(f"🔗 Creating temporary shortcut: {temp_shortcut_name}")

    create_shortcut(drive_service, folder_id, temp_shortcut_name)

    source_path = f"/content/drive/MyDrive/{temp_shortcut_name}"
    destination_path = f"{BASE_PATH}/{DESTINATION_FOLDER_NAME}"

    # Waiting a bit until the shortcut syncs
    max_retries = 10
    retry_delay_seconds = 5
    shortcut_found = False
    print("Waiting for file system sync...")
    for i in range(max_retries):
        if os.path.isdir(source_path):
            shortcut_found = True
            break
        print(f"Retry {i+1}/{max_retries}: Syncing... ({retry_delay_seconds}s)")
        time.sleep(retry_delay_seconds)

    if not shortcut_found:
        raise Exception("Shortcut created but not found. Try running again.")

    # 3. Copy Process
    if not os.path.exists(source_path):
        print(f"❌ Error: Source folder not found.")
    else:
        size_in_bytes, file_count = get_size_and_count(source_path)
        print(f"\n📂 Valid Files to Copy: {file_count} | Size: {size_in_bytes / (1024**3):.2f} GB")

        # Destination Folder Check (It's okay if it exists, otherwise create it)
        if not os.path.exists(destination_path):
            print(f"🚀 Creating new folder: {destination_path}")
            os.makedirs(destination_path)
        else:
            print(f"ℹ️ Folder already exists. Merging/Resuming...")

        if file_count == 0:
            print("ℹ️ No valid files found to copy.")
        else:
            pbar = tqdm(total=file_count, unit='file', desc='Syncing')

            skipped_existing = 0
            skipped_google = 0

            for dirpath, dirnames, filenames in os.walk(source_path):
                rel_path = os.path.relpath(dirpath, source_path)
                dest_dir = os.path.join(destination_path, rel_path)

                if not os.path.exists(dest_dir):
                    os.makedirs(dest_dir)

                for file in filenames:
                    src_file = os.path.join(dirpath, file)
                    dst_file = os.path.join(dest_dir, file)

                    # Check 1: Check if it's Google Docs/Slides
                    if file.lower().endswith(GOOGLE_WORKSPACE_EXTENSIONS):
                        skipped_google += 1
                        # These were not included in the total count, so no need to update pbar.
                        continue

                    # Check 2: Check if the file already exists
                    if os.path.exists(dst_file):
                        skipped_existing += 1
                        pbar.update(1) # If it exists, quickly update the progress bar.
                        continue

                    # Copying...
                    try:
                        shutil.copy2(src_file, dst_file)
                        pbar.update(1)
                    except OSError as e:
                        # If Errno 95 occurs (sometimes due to Google files without extensions)
                        if e.errno == 95 or "Operation not supported" in str(e):
                            skipped_google += 1
                            pbar.update(1)
                        else:
                            print(f"\n⚠️ Error copying {file}: {e}")

            pbar.close()
            print("\n✅ Copying Completed Successfully!")
            print(f"📊 Summary: Skipped (Existing): {skipped_existing} | Skipped (Google Files): {skipped_google}")

except Exception as e:
    print(f"\n❌ An error occurred: {e}")
    print("Note: If the link is 'Restricted', make sure you have access.")

finally:
    # 4. Delete the shortcut
    if temp_shortcut_name and os.path.exists(f"/content/drive/MyDrive/{temp_shortcut_name}"):
        print(f"🧹 Cleaning up temporary shortcut...")
        try:
            os.remove(f"/content/drive/MyDrive/{temp_shortcut_name}")
            print("✨ All Done!")
        except:
            print("⚠️ Could not delete shortcut automatically. You can delete it manually from Drive.")

Method 3: Direct API Copy (including Google Workspace files)

This method utilizes the Google Drive API directly to recursively copy files and folders from a shared Google Drive link to your My Drive. A key advantage of this approach is its ability to handle Google Workspace native files (like Google Docs, Sheets, Slides) by without converting them into standard formats during the copy process. This ensures a complete and usable backup of all content from the shared folder.

In [ ]:
import sys
import re
import time
import json
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# 1. Authenticate
print("Authenticating User...")
auth.authenticate_user()
service = build('drive', 'v3')

# ==========================================
#              CONFIGURATION
# ==========================================

# Source Link
SOURCE_FOLDER_LINK = "YOUR_SHARED_DRIVE_LINK_HERE"

# Path to save
DESTINATION_PATH = "Your Destination Folder/Subfolder"

# ==========================================

processed_file_ids = set()

def get_id_from_link(link):
    match = re.search(r'folders/([a-zA-Z0-9-_]+)', link)
    if match: return match.group(1)
    match = re.search(r'id=([a-zA-Z0-9-_]+)', link)
    if match: return match.group(1)
    return link

def get_or_create_folder_path(path_string):
    parts = path_string.strip("/").split("/")
    parent_id = 'root'
    for part in parts:
        query = f"name = '{part.replace("'", "\'")}' and '{parent_id}' in parents and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
        response = service.files().list(q=query, spaces='drive', fields='files(id)').execute()
        files = response.get('files', [])
        if files:
            parent_id = files[0]['id']
        else:
            file_metadata = {'name': part, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [parent_id]}
            folder = service.files().create(body=file_metadata, fields='id').execute()
            parent_id = folder.get('id')
    return parent_id

def is_file_name_exist(name, parent_id):
    safe_name = name.replace("'", "\'")
    query = f"name = '{safe_name}' and '{parent_id}' in parents and trashed = false"
    response = service.files().list(q=query, spaces='drive', fields='files(id)').execute()
    return len(response.get('files', [])) > 0

def copy_files_recursive(source_id, dest_id):
    query = f"'{source_id}' in parents and trashed = false"
    page_token = None

    while True:
        try:
            # List files in the current folder
            response = service.files().list(
                q=query,
                spaces='drive',
                fields='nextPageToken, files(id, name, mimeType)',
                pageToken=page_token
            ).execute()

            files = response.get('files', [])

            for file in files:
                file_id = file['id']
                file_name = file['name']

                # --- 1. ID Check (Loop Protection) ---
                if file_id in processed_file_ids:
                    continue

                # --- 2. Folder Handling ---
                if file['mimeType'] == 'application/vnd.google-apps.folder':
                    print(f"\n📂 Entering Folder: {file_name}")
                    sub_folder_id = get_or_create_folder_path_simple(file_name, dest_id)
                    processed_file_ids.add(file_id)
                    copy_files_recursive(file_id, sub_folder_id)

                # --- 3. File Handling ---
                else:
                    # Check if exists
                    if is_file_name_exist(file_name, dest_id):
                        sys.stdout.write(f"\r⏩ Skipped (Exists): {file_name[:30]}...           ")
                        sys.stdout.flush()
                        processed_file_ids.add(file_id)
                        continue

                    # --- COPY ATTEMPT (With Error Handling per File) ---
                    try:
                        sys.stdout.write(f"\rhg Copying: {file_name[:30]}...                     ")
                        sys.stdout.flush()

                        file_metadata = {
                            'name': file_name,
                            'parents': [dest_id]
                        }
                        service.files().copy(fileId=file_id, body=file_metadata).execute()
                        processed_file_ids.add(file_id)

                    except HttpError as error:
                        # Parse Error
                        error_reason = ""
                        try:
                            error_content = json.loads(error.content.decode('utf-8'))
                            error_reason = error_content['error']['errors'][0]['reason']
                        except:
                            pass

                        if error_reason == 'cannotCopyFile':
                            print(f"\n⛔ Skipped (Restricted by Owner): {file_name}")
                        else:
                            print(f"\n❌ Failed to copy {file_name}: {error_reason}")

            # Check for next page
            page_token = response.get('nextPageToken', None)
            if page_token is None:
                break

        except HttpError as error:
            print(f"\n⚠️ Batch Error (Retrying in 5s): {error}")
            time.sleep(5)
        except Exception as e:
            print(f"\n❌ Critical Error: {e}")
            break

def get_or_create_folder_path_simple(name, parent_id):
    safe_name = name.replace("'", "\'")
    query = f"name = '{safe_name}' and '{parent_id}' in parents and mimeType = 'application/vnd.google-apps.folder' and trashed = false"
    response = service.files().list(q=query, spaces='drive', fields='files(id)').execute()
    files = response.get('files', [])
    if files: return files[0]['id']
    else:
        file_metadata = {'name': name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [parent_id]}
        f = service.files().create(body=file_metadata, fields='id').execute()
        return f.get('id')

# --- Main Program ---
try:
    source_folder_id = get_id_from_link(SOURCE_FOLDER_LINK)
    if not source_folder_id: raise Exception("Link Error")

    print(f"\n🚀 Starting Copy Process (Skipping Restricted Files)...")
    final_dest_id = get_or_create_folder_path(DESTINATION_PATH)

    copy_files_recursive(source_folder_id, final_dest_id)

    print("\n\n✅ Done! (Check logs for skipped restricted files)")

except Exception as e:
    print(f"\n❌ Error: {e}")